## weight initialization
- weight 를 잘 초기화하지 않고 무작위로 하면 학습 성능이 떨어짐.
- 방법
1. 간단한 상수로 초기화 : 0으로 초기화 -> 안 좋은 방식 : 역전파 쓸 때 모든 gradient가 0이 될 수 있음.
2. challenging issue
3. RBM (Restricted Boltzmann Machine) 이용해 초기화하면 좋은 성능 

#### RBM
- 레이어 안에서의 노드들끼리는 연결이 없지만 다른 레이어 사이에서는 fully connected (다 연결)
- Forward (Encoding): 입력값 $x$가 들어왔을 때 가중치 $W$를 곱해 특징값 $y$(은닉층)를 생성합니다.
- Backward (Decoding): 은닉층의 $y$를 바탕으로 다시 입력값 $x$를 복원(Reconstruction)합니다.학습 목표: 입력된 $x$와 복원된 $x'$ 사이의 차이를 최소화하는 가중치 $W$를 찾는 것입니다.
- 가중치 초기화 법 : pretraining step

#### Deep Belief Network (DBN)
: DBN의 핵심: RBM을 벽돌 쌓듯 한 층씩 미리 학습시켜(Pre-training), 멍청한 상태가 아닌 어느 정도 똑똑해진 상태에서 전체 학습(Fine-tuning)을 시작하는 것.
- fine tuning : 사전 학습(Pre-training)을 통해 얻은 초기 가중치를 바탕으로, 실제 정답 데이터를 사용하여 전체 신경망의 가중치를 목적에 맞게 미세하게 조정하는 단계
- 여러 층의 RBM을 쌓아 올리면서 한 층씩 미리 학습(Pre-training)시킨 뒤, 여기서 얻은 가중치를 심층 신경망(DNN)의 초기값으로 사용합니다.

## Xavier / He initialization
: 무작위로 초기화 하는 게 아니라 레이어 특성에 따라 다르게 초기화 
- 수식이 존재
- 샤비에 초기화 법 : 이전 층과 다음 층의 노드 수에 맞춰 가중치 분산을 조절함으로써, 활성화 함수가 Sigmoid나 Tanh일 때 데이터가 0이나 1로 쏠리는 현상을 방지
- 헤 초기화 법 : Xavier 초기화의 원리를 발전시켜 분산을 2배로 늘린 방식이며, 주로 ReLU 활성화 함수를 사용할 때 신호가 사라지는(Dead ReLU) 문제를 해결하는 데 최적화
1. xavier normal initialization 
2. xavier uniform initialization
##### 장점 : RBM처럼 별도의 복잡한 사전 학습(Pre-training) 단계 없이 간단한 수학적 계산만으로도 깊은 신경망의 가중치를 안정적으로 초기화하여 학습을 바로 시작할 수 있음.

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100


# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

# nn layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [2]:
# 샤비에 초기화!
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.0215, -0.0894,  0.0598,  ...,  0.0200,  0.0203,  0.1212],
        [ 0.0078,  0.1378,  0.0920,  ...,  0.0975,  0.1458, -0.0302],
        [ 0.1270, -0.1296,  0.1049,  ...,  0.0124,  0.1173, -0.0901],
        ...,
        [ 0.0661, -0.1025,  0.1437,  ...,  0.0784,  0.0977, -0.0396],
        [ 0.0430, -0.1274, -0.0134,  ..., -0.0582,  0.1201,  0.1479],
        [-0.1433,  0.0200, -0.0568,  ...,  0.0787,  0.0428, -0.0036]],
       requires_grad=True)

In [3]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.246781081
Epoch: 0002 cost = 0.093050748
Epoch: 0003 cost = 0.061956182
Epoch: 0004 cost = 0.044412050
Epoch: 0005 cost = 0.032645479
Epoch: 0006 cost = 0.023976512
Epoch: 0007 cost = 0.022737484
Epoch: 0008 cost = 0.018594194
Epoch: 0009 cost = 0.014644416
Epoch: 0010 cost = 0.015323286
Epoch: 0011 cost = 0.013379679
Epoch: 0012 cost = 0.011164513
Epoch: 0013 cost = 0.009801405
Epoch: 0014 cost = 0.011504559
Epoch: 0015 cost = 0.009029884
Learning finished


##### weight 초기화 방식만 바꿔도 성능 높일 수 있음!!

In [ ]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

## mnist_nn_deep

In [ ]:
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4, relu, linear5).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Parameter containing:
tensor([[ 0.0579, -0.1055,  0.0272,  ...,  0.1036,  0.0291, -0.0746],
        [ 0.0528,  0.0494, -0.0059,  ..., -0.0951, -0.0887, -0.0129],
        [-0.0953, -0.0510,  0.1030,  ..., -0.0493, -0.0581, -0.0280],
        ...,
        [ 0.0631,  0.0917,  0.0690,  ..., -0.0746, -0.0798,  0.0231],
        [ 0.0937,  0.0521, -0.0346,  ...,  0.0432, -0.0982, -0.0107],
        [-0.0888, -0.0723,  0.0511,  ...,  0.0530, -0.0830,  0.0624]],
       requires_grad=True)